# Self-Driving Car Engineer Nanodegree

## Project: Build a Traffic Sign Recognition Classifier

In this project, we will use what you've learned about deep neural networks and convolutional neural networks to classify traffic signs. Specifically, you'll train a model to classify traffic signs from the German Traffic Sign Dataset.

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

In [0]:
# Import Dependencies
import pandas as pd
import numpy as np
import csv
import random
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.contrib.layers import flatten
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
#---------------------------------------------------------------

---
## Step 0: Load The Data

In [0]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
validation_file='traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [0]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [0]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [0]:
# Loop over range
for index in range(100,101):
    image = X_train[index]
    plt.figure(figsize=(1,1))
    plt.imshow(image)
    print(y_train[index])

In [0]:
# In this block we will write a function to visualize and show image

%matplotlib inline
class_id_to_sign = {}
# Assign Sementics (class_id_to_sign)
with open('signnames.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        class_id_to_sign[row[0]] = row[1]

# Function for Visulaizing Images
## Inputs: Images and Labels (Respectively)  
def VisualizeImages(images, labels):
    n_columns = 8   #Number_of_columns
    n_rows = int(len(images)/n_columns)+1   #number_of_rows
    
    # Adjust Width and Height of Displayed Image(s)
    width = 24
    height = n_rows * 3
    
    # Define
    fig, ax = plt.subplots(n_rows,n_columns, figsize=(width, height))
    fig.subplots_adjust(hspace = .2, wspace=.1)
    ax = ax.ravel()
    for i in range(n_columns * n_rows):
        ax[i].axis('off')
        if (i<len(images)):
            image = images[i]
            ax[i].axis('off')
            #if (image.shape[2] == 1):
            #    print(image)
            #    ax[i].imshow(image.astype(np.uint8).squeeze, cmap='gray')
            #else:

            # Display the Respective Image
            ax[i].imshow(image)
            # Set Title according to Label
            ax[i].set_title('{} ({:.20})'.format(labels[i], class_id_to_sign[str(labels[i])]))
 #----------------------------------------------------------------------------------------

In [0]:
# This block will draw the images by utilizing the above defined visualization function

# Declaring two arrays, one for images, one for labels
X = []
Y = []

# Assigning Labels and Indices
labels, indices = np.unique(y_train, return_index=True)

# Iteration
for i in range(len(labels)):
    X.append(X_train[indices[i]])
    Y.append(labels[i])

# Call Function 'VisualizeImage' to Draw Images over the setted grid
VisualizeImages(X,Y)
#---------------------------------------------------------------------------

In [0]:
# Now we make a histogram for the distribution Using Bar Graph (Built in Numpy)
x,y = np.unique(y_train, return_counts=True)
plt.bar(x,y)
plt.show
#--------------------------------------------------------------------------

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [0]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# We need to implement grayscale function as well as normalization function for images
def PreProcessImages(color_images):
    # Grayscale Function
    grayscaled_images = np.sum(color_images/3, axis=3, keepdims=True)
    
    # Function for Normalization of Images
    normalized_images = (grayscaled_images - 128) / 128
      
    return normalized_images  #return the pre_proccessed images

In [0]:
# Now we define a function to get a random image
def GetRandomImage(x, y, filter_index):
    indices, = np.nonzero( y==filter_index )
    index = np.random.choice(indices)
    return x[index]
    
# Draw a Random Image
image = GetRandomImage(X_train, y_train, 23)  #change this 23
plt.figure(figsize=(1,1))
plt.imshow(image)

print (X_train.shape)
pre_processed = PreProcessImages(X_train)
print (pre_processed.shape)

### Model Architecture

In [0]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Here, the exciting part is that we will be playing out with tensors.

# Define Epochs number, batch size and drop_out rate
## Change em
EPOCHS = 150
BATCH_SIZE = 128
DROPOUT = 0.5

#Shuffle Data
X_train, y_train = shuffle(X_train, y_train)

# Here we define different placeholder variables
# A placeholder is simply a variable that we will assign data to at a later time.
keep_prob = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43) # Returns a one-hot tensor  (one-hot Encode y)
keep_prob = tf.placeholder(tf.float32)

In [0]:
# Here, we Implement the LeNet-5
# As show in Udacity's Lectures in the classroom
# Taking CNN Lesson as a basic starting point
# It is kind of plug and play

def LeNet(x):    

    # First we define the Hyperparameters.
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.

    # Weight
    conv_weight_1 = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    
    # Bias
    conv_bias_1 = tf.Variable(tf.zeros(6))

    # Now, we apply convolution as:
    layer_1 = tf.nn.conv2d(x, conv_weight_1, strides=[1, 1, 1, 1], padding='VALID') + conv_bias_1
    # Activating (Layer 1) as:
    layer_1 = tf.nn.relu(layer_1)
    # Pooling (Layer 1). Input = 28x28x6. Output = 14x14x6.
    layer_1 = tf.nn.max_pool(layer_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')


    # Layer 2: Convolutional. Output = 10x10x16.
    # Weight
    conv_weight_2 = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    
    # Bias
    conv_bias_2 = tf.Variable(tf.zeros(16))
    
    # Now, we apply convolution (second layer) as:
    layer_2   = tf.nn.conv2d(layer_1, conv_weight_2, strides=[1, 1, 1, 1], padding='VALID') + conv_bias_2
    # Activating (Layer 2) as:
    layer_2 = tf.nn.relu(layer_2)
    # Pooling (Layer 2). Input = 10x10x16. Output = 5x5x16.
    layer_2 = tf.nn.max_pool(layer_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten (Layer 2). Input = 5x5x16. Output = 400.
    fully_connected_0   = flatten(layer_2)
    fully_connected_0   = tf.nn.dropout(fully_connected_0, keep_prob)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    # This layer is fully connected. 
    # Weight
    fully_connected_weight_1 = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    
    # Bias
    fully_connected_bias_1 = tf.Variable(tf.zeros(120))

    # Here, instead of 2D Convolution, we are using matrix multiplication
    fully_connected_1   = tf.matmul(fully_connected_0, fully_connected_weight_1) + fully_connected_bias_1
    # Activating (Layer 3) as:
    fully_connected_1    = tf.nn.relu(fully_connected_1)
    fully_connected_1    = tf.nn.dropout(fully_connected_1, keep_prob) 

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    # Weight
    fully_connected_weight_2  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    
    # Bias
    fully_connected_bias_2  = tf.Variable(tf.zeros(84))

    # Here, instead of 2D Convolution, we are using matrix multiplication
    fully_connected_2    = tf.matmul(fully_connected_1, fully_connected_weight_2) + fully_connected_bias_2 
    # Activating (Layer 4) as:
    fully_connected_2    = tf.nn.relu(fully_connected_2)
    fully_connected_2    = tf.nn.dropout(fully_connected_2, keep_prob) 


    # Layer 5: Fully Connected. Input = 84. Output = 43.
    # As, this is the final layer, so our output need to be matched up with the number of classes
    fully_connected_weight_3  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fully_connected_bias_3  = tf.Variable(tf.zeros(43))

    # Matrix Multiplication
    logits = tf.matmul(fully_connected_2, fully_connected_weight_3) + fully_connected_bias_3
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [0]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

# Setting Learning Rate
rate = 0.0005

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)

# Set loss functions and optimizer
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

# Validation & Evaluation (of Neural Network)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def Evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

X_train_preProcessed = PreProcessImages(X_train)
X_valid_preProcessed = PreProcessImages(X_valid)
X_test_preProcessed  = PreProcessImages(X_test)


In [0]:
# Training Block
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    validation_accuracies = []    # Accuracies for validation
    
    # Iteration Starts
    for i in range(EPOCHS):
        X_train, X_train_preProcessed, y_train = shuffle(X_train, X_train_preProcessed, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_preProcessed[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: DROPOUT})
            
        training_accuracy = evaluate(X_train_preProcessed, y_train)
        validation_accuracy = evaluate(X_valid_preProcessed, y_valid)
        
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        validation_accuracies.append(validation_accuracy)
    
    # Save Model
    save_file = 'model.ckpt'
    saver.save(sess, save_file)
    print("Model saved")
    
    # Plot Statistics (Model)
    plt.plot(range(EPOCHS),validation_accuracies)
    plt.show

In [0]:
# Now, we launch the model on our test data
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = Evaluate(X_test_preProcessed, y_test)
    #test_accuracy = sess.run(accuracy_operation, feed_dict={x: X_test, y: y_test, keep_prob : 1.0})

    print("Test Accuracy = {:.3f}".format(test_accuracy))
#----------------------------------------------------------------

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [0]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

traffic_signs_data_folder = 'test_data/'
  
my_X_test = []
my_Y_test = []
for image_file in glob.glob(traffic_signs_data_folder + "*.jpg"):
    image = np.array(mpimg.imread(image_file))
    try:
        label_str = image_file[len(traffic_signs_data_folder):][:2]
        label = int(label_str)
        my_X_test.append(image)
        my_Y_test.append(label)
    except  Exception:
        print ('Ignoring image {}'.format(image_file))
        
VisualizeImages(my_X_test, my_Y_test)

### Predict the Sign Type for Each Image

In [0]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

X_test_PreProcessed = PreProcess(np.array(my_X_test))
prediction=tf.argmax(logits,1)  # Prediction

with tf.Session() as sess:
    saver.restore(sess, save_file)
    predictions = sess.run(prediction, feed_dict={x: X_test_PreProcessed, keep_prob: 1.0})
    visualizeImages(my_X_test, predictions)
    # Print Predictions
    print(predictions)

### Analyze Performance

In [0]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, save_file)
    # Test Accuracy
    test_accuracy = evaluate(my_X_test_preProcessed, my_Y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [0]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

def VisualizeTopk1(test_images, train_images, top_k):
    probabilities = top_k.values
    indices = top_k.indices
    #assert (len(test_images) == len(indices))
    
    # Again Same Procedure, (Defining Columns, Rows, Width, Height)
    n_columns = 6
    n_rows = len(indices)
    width = 24
    height = n_rows * 3
    fig, ax = plt.subplots(n_rows,n_columns, figsize=(width, height))
    fig.subplots_adjust(hspace = .2, wspace=.1)
    ax = ax.ravel()
    
    i = 0
    for row in range(n_rows):
        ax[i].axis('off')
        ax[i].imshow(test_images[row])
        #axs[i].set_title('{} ({:.20})'.format(labels[i], class_id_to_sign[str(labels[i])]))
        i += 1
        for j in range(len(indices[row])):
            index = indices[row][j]
            probability = probabilities[row][j]
            ax[i].axis('off')
            ax[i].set_title('{:.2f}% ({} {:.20})'.format(probability*100, index,  class_id_to_sign[str(index)]))
            ax[i].imshow(GetRandomImage(X_train, y_train, index))
            i += 1

In [0]:
def visualizeTopk2(test_images, train_images, top_k):
    probabilities = top_k.values
    indices = top_k.indices
    #assert (len(test_images) == len(indices))
          
    # Again Same Procedure, (Defining Columns, Rows, Width, Height)
    n_columns = 2
    n_rows = len(indices)
    width = 24
    height = n_rows * 3

    fig, ax = plt.subplots(n_rows,n_columns, figsize=(width, height))
    fig.subplots_adjust(hspace = .2, wspace=.1)
    ax = ax.ravel()

    i = 0
    for row in range(n_rows):
        ax[i].axis('off')
        ax[i].imshow(test_images[row])
        i += 1
        labs=[class_id_to_sign[str(j)] for j in indices[row]]
        ax[i].barh(labs, probabilities[row])
        
        #ax[i].yticks(np.arange(1, 6, 1), labs)
        i += 1

In [0]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

softmax_logits = tf.nn.softmax(logits)
top_k=tf.nn.top_k(softmax_logits,5)

with tf.Session() as sess:
    saver.restore(sess, save_file)
    top_k = sess.run(top_k, feed_dict={x: X_test_PreProcessed, keep_prob : 1.0})
    
    VisualizeTopk1(my_X_test, X_train, top_k)
    VisualizeTopk2(my_X_test, X_train, top_k)
  #-----------------------------------------------------------------------------

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [0]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")